In [13]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import time
import re
# from screeninfo import get_monitors
import pandas as pd
import random
import os
import ast

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# US

In [14]:
#replace as needed
folder_name = "Supplementary Materials"
followers_path = os.path.join(folder_name, "followers_US.txt")
completed_path = os.path.join(folder_name, "completed_accounts")

followers_US = []
# with open(r'''X\followers_US.txt''', 'r') as f: # PC
with open(followers_path,'r') as f: #List of accounts from raw followers list
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            if follower not in followers_US:
                # print(follower)
                followers_US.append(follower)
f.close()

# with open(completed_path, 'r') as f: #Get list of accounts that are completed
#     for line in f:
#         if line.startswith('total'):
#             completed_list = ast.literal_eval(line[8:])

#Filter out the ones that are completed
# followers_US_set = set(followers_US)
# followers_US = [f for f in followers_US_set if f not in completed_list]

print(followers_US)

print(f"Number of followers to process: {len(followers_US)}")

['ThomasEWoods', 'RealSpikeCohen', 'ComicDaveSmith', 'FreemansMind96', 'HannahDCox', 'checkmatestate', 'SabbySabs2', 'MsLaToshaBrown', 'RonFilipkowski', 'KyleKulinski', 'funder', 'mmpadellan', 'krystalball', 'SteveSchmidtSES', 'robreiner', 'marceelias', 'DavidAFrench', 'BillOReilly', 'jimmy_dore', 'TheRickWilson', 'davidsirota', 'gtconway3d', 'spann', 'Jaybefaunt', 'ProudSocialist', 'BernardKerik', 'jeff_poor', 'TomFitton', 'brad_polumbo', 'IngrahamAngle', 'seanhannity', 'charliekirk11', 'TuckerCarlson', 'AnnCoulter', 'EricTrump', 'LaraLeaTrump', 'PatTheSocialist', 'TristanSnell', 'NancyAFrench', 'donwinslow', 'KyleClark', 'BobMurphyEcon', 'bshelburne', 'PatrickSvitek', 'evanasmith', 'BarbMcQuade', 'JohnArchibald', 'natsechobbyist', 'davidhogg111', 'anthonyzenkus', 'benshapiro', 'briebriejoy', 'fred_guttenberg', 'LeeCamp', 'JordanChariton', 'kylegriffin1', 'elonmusk', 'itsJeffTiedrich', 'bluestein', 'SarcasmStardust', 'SocialistMMA', 'Josh_Moon', 'WarOnDumb', 'toddcstacy', 'maryltrump'

In [15]:
tweets_dict_US = {}

# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

In [16]:
# driver = webdriver.Chrome()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


In [17]:
### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys("blaketrn@udel.edu")
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys("Idontevenknow1!")
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys("HaHoangNha55719")
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys("Idontevenknow1!")
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010be66f88 chromedriver + 7110536
1   chromedriver                        0x000000010be5ef8a chromedriver + 7077770
2   chromedriver                        0x000000010b8000f0 chromedriver + 397552
3   chromedriver                        0x000000010b84c383 chromedriver + 709507
4   chromedriver                        0x000000010b84c681 chromedriver + 710273
5   chromedriver                        0x000000010b891e14 chromedriver + 994836
6   chromedriver                        0x000000010b87093d chromedriver + 858429
7   chromedriver                        0x000000010b88f234 chromedriver + 983604
8   chromedriver                        0x000000010b8706b3 chromedriver + 857779
9   chromedriver                        0x000000010b83f182 chromedriver + 655746
10  chromedriver                        0x000000010b84015e chromedriver + 659806
11  chromedriver                        0x000000010be2c3d0 chromedr

In [ ]:
count = 1

keyword1 = "(Gaza* OR Israel* OR (West Bank) OR (Tel Aviv) OR (Tel Aviv-Yafo) OR Bedouin* OR Palestin* OR Middle OR East* OR Middle OR Eastern OR Jew* OR rabbi OR Muslim* OR Islam* OR Jihad OR Antisemiti* OR Zionis* OR IDF)"
keyword2 = "(IOF OR Hamas OR Massacre OR Genocide OR Ceasefire OR Terroris* OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif OR Rafah OR Khan OR Younis OR Ben-Gvir OR Abbas OR Gallant)"
hashtags = "(# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide)"
until = "2023-10-06"
since = "2022-10-07"

unprocessed = []

for profile in tqdm(followers_US):
    if count%5 == 0:
        print("5 Accounts passed, sleeping for 5 minutes.")
        time.sleep(300)
    
    if profile not in tweets_dict_US.keys():
        print(profile)

        tweets_dict_US[profile] = []

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        for i in range(2): 
            search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]')))

            # Advanced search query
            if i == 0:
                search_query.send_keys(f'{keyword1} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')
            if i == 1:
                search_query.send_keys([Keys.BACKSPACE]*1000)
                search_query.send_keys(f'{keyword2} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')

            search_query.send_keys(Keys.RETURN)
            time.sleep(6)
            
            # Scroll until you cannot scroll anymore
            reached_page_end = False
            last_height = driver.execute_script("return document.body.scrollHeight")

            retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
            # random.randrange(10, 15)
            if (is_element_present(driver, retry_xpath)):
                try:
                    print(f'{profile} - {count} - Retry button appeared.')
                    unprocessed.append(profile)
                    for i in range(5):
                        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                        search_button.click()
                    time.sleep(100) 
                except:
                    print(f"{profile} - Failed to click Retry")

            while not reached_page_end:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
                time.sleep(4)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if last_height == new_height:
                    reached_page_end = True
                else:
                    last_height = new_height
            try:
                # Select the home timeline div
                search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
                # Just get all the HTML, will parse later
                tweets_listed = search_results.get_attribute('innerHTML')
                time.sleep(3)

                # Append to dict
                tweets_dict_US[profile].append(tweets_listed)

            except:
                pass

    count+=1

print("Unprocessed accounts for this run:")
for profile in unprocessed:
    print(f"@{profile}\n")


  0%|          | 0/75 [00:00<?, ?it/s]

ThomasEWoods


  1%|▏         | 1/75 [01:22<1:41:42, 82.46s/it]

RealSpikeCohen


  3%|▎         | 2/75 [02:17<1:20:28, 66.15s/it]

...

FiorellaIsabelM


 99%|█████████▊| 74/75 [2:06:50<01:12, 72.95s/it]

5 Accounts passed, sleeping for 5 minutes.
aaronjmate


100%|██████████| 75/75 [2:12:38<00:00, 106.11s/it]

Unprocessed accounts for this run:


In [ ]:
# driver.quit()

In [ ]:
tweets_dict_US.keys()

dict_keys(['ThomasEWoods', 'RealSpikeCohen', 'ComicDaveSmith', 'FreemansMind96', 'HannahDCox', 'checkmatestate', 'SabbySabs2', 'MsLaToshaBrown', 'RonFilipkowski', 'KyleKulinski', 'funder', 'mmpadellan', 'krystalball', 'SteveSchmidtSES', 'robreiner', 'marceelias', 'DavidAFrench', 'BillOReilly', 'jimmy_dore', 'TheRickWilson', 'davidsirota', 'gtconway3d', 'spann', 'Jaybefaunt', 'ProudSocialist', 'BernardKerik', 'jeff_poor', 'TomFitton', 'brad_polumbo', 'IngrahamAngle', 'seanhannity', 'charliekirk11', 'TuckerCarlson', 'AnnCoulter', 'EricTrump', 'LaraLeaTrump', 'PatTheSocialist', 'TristanSnell', 'NancyAFrench', 'donwinslow', 'KyleClark', 'BobMurphyEcon', 'bshelburne', 'PatrickSvitek', 'evanasmith', 'BarbMcQuade', 'JohnArchibald', 'natsechobbyist', 'davidhogg111', 'anthonyzenkus', 'benshapiro', 'briebriejoy', 'fred_guttenberg', 'LeeCamp', 'JordanChariton', 'kylegriffin1', 'elonmusk', 'itsJeffTiedrich', 'bluestein', 'SarcasmStardust', 'SocialistMMA', 'Josh_Moon', 'WarOnDumb', 'toddcstacy', 'm

In [ ]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

dict_keys(['ThomasEWoods', 'RealSpikeCohen', 'ComicDaveSmith', 'FreemansMind96', 'HannahDCox', 'checkmatestate', 'SabbySabs2', 'MsLaToshaBrown', 'RonFilipkowski', 'KyleKulinski', 'funder', 'mmpadellan', 'krystalball', 'SteveSchmidtSES', 'robreiner', 'marceelias', 'DavidAFrench', 'BillOReilly', 'jimmy_dore', 'TheRickWilson', 'davidsirota', 'gtconway3d', 'spann', 'Jaybefaunt', 'ProudSocialist', 'BernardKerik', 'jeff_poor', 'TomFitton', 'brad_polumbo', 'IngrahamAngle', 'seanhannity', 'charliekirk11', 'TuckerCarlson', 'AnnCoulter', 'EricTrump', 'LaraLeaTrump', 'PatTheSocialist', 'TristanSnell', 'NancyAFrench', 'donwinslow', 'KyleClark', 'BobMurphyEcon', 'bshelburne', 'PatrickSvitek', 'evanasmith', 'BarbMcQuade', 'JohnArchibald', 'natsechobbyist', 'davidhogg111', 'anthonyzenkus', 'benshapiro', 'briebriejoy', 'fred_guttenberg', 'LeeCamp', 'JordanChariton', 'kylegriffin1', 'elonmusk', 'itsJeffTiedrich', 'bluestein', 'SarcasmStardust', 'SocialistMMA', 'Josh_Moon', 'WarOnDumb', 'toddcstacy', 'm

In [ ]:
raw_data_folder = os.path.join("Raw Data", "After")
raw_json = os.path.join(raw_data_folder, f"US_tweets_{until}_{since}.json")
cleaned_json = os.path.join(raw_data_folder, f"US_tweets_cleaned_{until}_{since}.json")

In [ ]:
with open(raw_json, 'w') as file:
    json.dump(tweets_dict_US, file)

In [ ]:
with open(cleaned_json, 'w') as file:
    json.dump(tweets_dict_US_cleaned, file)